<a href="https://colab.research.google.com/github/DataLDana/ABTesting/blob/main/EniacAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from scipy.stats import chi2_contingency

# import data
url = "https://drive.google.com/file/d/1FaKyKiJORoHS9WhlLS3m9Rt0UY4kPjfm/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
a = pd.read_csv(path)

url = "https://drive.google.com/file/d/1WIbFJUnEZCEM0z5V58bafKWUVGnTs18J/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
b = pd.read_csv(path)

url = "https://drive.google.com/file/d/10XFiJgh4eBVZd5Jde_Srn33Ct8NubzDt/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
c = pd.read_csv(path)

url = "https://drive.google.com/file/d/1dRYBKEjtsIU4LwdQAiT69oNqyqaeHWxh/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
d = pd.read_csv(path)

In [ ]:
# get number of clicks
num_visitors = []
import regex as re
# a                                                              # a.loc[1,'Snapshot information'] = 'created 2021-09-14   •   14 days 0 hours 34 mins   •   25326 visits, 23174 clicks'
txt = re.findall('\d* visits',a.loc[1,'Snapshot information'])   # returns ['25326 visits']
number_str = txt[0].split(' visits')[0]                          # only takes first entry in txt as a string and splits number and 'visits'
num_visitors.append(number_str)                                  # save it to the list
# b
txt = re.findall('\d* visits',b.loc[1,'Snapshot information'])
number_str = txt[0].split(' visits')[0]
num_visitors.append(number_str)
# c
txt = re.findall('\d* visits',c.loc[1,'Snapshot information'])
number_str = txt[0].split(' visits')[0]
num_visitors.append(number_str)
# d
txt = re.findall('\d* visits',d.loc[1,'Snapshot information'])
number_str = txt[0].split(' visits')[0]
num_visitors.append(number_str)
#
num_visitors

['25326', '24747', '24876', '25233']

In [ ]:
pd.set_option("display.max_colwidth", 100)

In [ ]:
# set version of test
a['Version'] = 'A'   # white shop now
b['Version'] = 'B'   # red shop now
c['Version'] = 'C'   # white SEE DEALS
d['Version'] = 'D'   # red SEE DEALS

# concatenate all 4 df into one
abcd = pd.concat([a,b,c,d],axis=0)

In [ ]:
abcd.sample(10)

,Element ID,Tag name,Name,No. clicks,Visible?,Snapshot information,Version
13,90,a,Services,177,True,NaN,A
1,25,div,mySidebar,305,True,"created 2021-10-27 • 14 days 0 hours 34 mins • 25233 visits, 23062 clicks",D
48,53,a,Help,37,True,NaN,D
37,119,input,Subject,201,True,NaN,A
30,125,img,magnetic-case,884,True,NaN,C
27,108,img,cartoon-case,861,True,NaN,C
13,90,a,Services,163,True,NaN,D
4,105,a,Accessories,1235,True,NaN,A
48,53,a,Help,39,True,NaN,B
22,72,img,iphone13_cover,890,True,NaN,C


In [ ]:
int(num_visitors[1])

24747

In [ ]:
# create a empty data frame
data = pd.DataFrame(index=['Type','Clicks', 'NoClicks','Visitors','CTR','Increase'],columns=['A','B','C','D'])
types = ['White SHOP NOW','Red SHOP NOW','White SEE DEALS','Red SEE DEALS']
# loop through four versions
versions = ['A','B','C','D']
i=0
# abcd = concatenated df of all data
for version in versions:
  data.loc['Type',version] = types[i]

  # calculate clicks on button
  mask = (abcd['Name'] == 'SHOP NOW') | (abcd['Name'] == 'SEE DEALS')                              # mask for button shop now OR see deals
  clicks =  abcd.loc[ mask & (abcd['Version'] == version), 'No. clicks'].iloc[0]                   # sum of click on BUTTON     OR USE .item() instead of .iloc[0]

  # write into df
  data.loc['Clicks',version] = clicks
  data.loc['NoClicks',version] = int(num_visitors[i]) - clicks                              # no clicks is Visitors - Clicks
  data.loc['Visitors',version] = int(num_visitors[i])
  data.loc['CTR',version] = clicks / int(num_visitors[i])
  data.loc['Increase',version] = (data.loc['CTR',version]/data.loc['CTR','A']) - 1                                # increase to the current CTR of 2%

  i+=1

data

,A,B,C,D
Type,White SHOP NOW,Red SHOP NOW,White SEE DEALS,Red SEE DEALS
Clicks,512,281,527,193
NoClicks,24814,24466,24349,25040
Visitors,25326,24747,24876,25233
CTR,0.020216,0.011355,0.021185,0.007649
Increase,0.0,-0.438331,0.047917,-0.621658


Questions
* What was the click-through rate for each version?
* Which version was the winner?
* Do the results seem conclusive?

Information:
* Null Hypothesis: all versions have the same CTR.
* Alternative Hypothesis: there is a difference in the CTR for the different versions.
$\alpha$ = 0.05
MDE = 20%


In [ ]:
observation = data.loc[['Clicks','NoClicks']]
observation

,A,B,C,D
Clicks,512,281,527,193
NoClicks,24814,24466,24349,25040


In [ ]:
alpha = 0.05
chisq, pvalue, df, expected = chi2_contingency(data.loc[['Clicks','NoClicks']])
print(chisq)
print(pvalue)
print(df)
if pvalue > alpha:
  print('No rejection of null Hypothesis.the difference in CTRs are likely to be by chance')
else:
  print('Rejection of Ho. at least one CTR is different')

224.01877488058412
2.7161216607868712e-48
3
Rejection of Ho. at least one CTR is different


# Post hoc test

* Version A - Version B
* Version A - Version C
* Version A - Version D
* Version B - Version C
* Version B - Version D
* Version C - Version D

In [ ]:
post_hoc_alpha = alpha/6   # make sure were sure, bonforoni adjustment
post_hoc_alpha

0.008333333333333333

In [ ]:
tests = [['A','B'],['A','C'],['A','D'],['B','C'],['B','D'],['C','D']]

In [ ]:
observation.loc[:,tests[0]]

,A,B
Clicks,512,281
NoClicks,24814,24466


In [ ]:
tests = [['A','B'],['A','C'],['A','D'],['B','C'],['B','D'],['C','D']]

for i in range(len(tests)):
  chisq, pvalue, df, expected = chi2_contingency(observation.loc[:,tests[i]])
  if pvalue > post_hoc_alpha:
    print('For Version', tests[i], ' the difference in CTRs are likely to be by chance')
  else:
    print('For Version', tests[i], ' the difference in CTRs are real')

For Version ['A', 'B']  the difference in CTRs are real
For Version ['A', 'C']  the difference in CTRs are likely to be by chance
For Version ['A', 'D']  the difference in CTRs are real
For Version ['B', 'C']  the difference in CTRs are real
For Version ['B', 'D']  the difference in CTRs are real
For Version ['C', 'D']  the difference in CTRs are real


In [ ]:
data

,A,B,C,D
Type,White SHOP NOW,Red SHOP NOW,White SEE DEALS,Red SEE DEALS
Clicks,512,281,527,193
NoClicks,24814,24466,24349,25040
Visitors,25326,24747,24876,25233
CTR,0.020216,0.011355,0.021185,0.007649
Increase,0.0,-0.438331,0.047917,-0.621658


- SHOP NOW VS SEE DEALS: no clear decision, for red shop now seems a bit better
- white vs red: red button performs worse!